In [1]:
import numpy as np
import math

class PDESolver:
    """ Abstract class to solve Black-Scholes PDE """

    def __init__(self, pde, imax, jmax):
        """ Constructor
        Parameters:
            pde : PDE to be solved
            imax : last value of the first variable's discretisation
            jmax : last value of the second variable's discretisation
        """
        # Initialising self variables
        self.pde  = pde
        self.imax = imax
        self.jmax = jmax
        
        # Initialising dt and dx 
        self.dt = pde.tau / imax
        self.dx = (pde.x_up - pde.x_low) / jmax

        # Initialising grid
        self.grid = np.empty((imax + 1, jmax + 1), dtype = float)

    def t(self, i):
        """ Return the descretised value of t at index i """
        return self.dt * i
    
    def x(self, j):
        """ Return the descretised value of x at index j """
        return self.dx * j

    # Helper umbrella function to get coefficients
    def a(self, i, j): return self.pde.get_coefficients(self.t(i), self.x(j))['a']
    def b(self, i, j): return self.pde.get_coefficients(self.t(i), self.x(j))['b']
    def c(self, i, j): return self.pde.get_coefficients(self.t(i), self.x(j))['c']
    def d(self, i, j): return self.pde.get_coefficients(self.t(i), self.x(j))['d']
    
    # Helper umbrella function to get boundary conditions
    def t_up(self, j): return self.pde.get_conditions(0, self.x(j))['tau']
    def x_low(self, i): return self.pde.get_conditions(self.t(i), 0)['x_low']
    def x_up(self, i): return self.pde.get_conditions(self.t(i), 0)['x_up']


    def interpolate(self, t, x):
        """ Get interpolated solution value at given time and space
        Parameters:
            t : point in time  
            x : point in space
        Return
            interpolated solution value
        
        NOTE:
        To interpolate and find value at point p (t, x), we must first find the discrete points
        i and j. Afterwards calculate the vertical and horizontal distance between the points;
               vertical distance = (t - i*dt)
               horizontal distance = (x - j*dx)
        Compute weighted average afterwards.


        (i+1, j) --------------- (i+1, j+1)
               |            |    |
               | ---------- p -- |
               |            |    |
               |            |    |
               |            |    |
               |            |    |
           (i,j) --------------- (i, j+1)
        """
        # Step 1: calculating closest discrete point (i, j), using floor division "//"
        i = int(t // self.dt) 
        j = int((x - self.pde.x_low) // self.dx)

        # Step 2: calculating vertical and horizontal distance/weight from each node
        #         (Since we are calculating weighting rather than physical distance
        #          we will divide it by dt or dx)
        i_1 = (t - self.dt * i) / self.dt
        i_0 = 1 - i_1
        j_1 = (x + self.pde.x_low - self.dx * j) / self.dx
        j_0 = 1 - j_1

        # Step 3: Returning the weighted average
        return (i_0 * j_0 * self.grid[i,j] 
                + i_1 * j_0 * self.grid[i+1, j]
                + i_0 * j_1 * self.grid[i, j+1] 
                + i_1 * j_1 * self.grid[i+1, j+1])


class ExplicitScheme(PDESolver):
    """ Black Scholes PDE solver using the explicit scheme
    """
    def __init__(self, pde, imax, jmax):
        super().__init__(pde, imax, jmax)


    # Functions for calculating coefficients
    """ Coefficient {*insert coefficient letter}_{i,j} for explicit scheme
    Parameters:
        i : index of x discretisation
        j : index of t discretisation
    Return:
        Desired coefficient
    """
    def A(self, i, j):
        return (self.dt / self.dx) * ((self.b(i, j) / 2) - (self.a(i, j) / self.dx))
    
    def B(self, i, j):
        return 1 - self.dt * self.c(i, j) + 2 * (self.dt * self.a(i, j) / (self.dx ** 2))
    
    def C(self, i, j):
        return - (self.dt / self.dx) * ((self.b(i, j) / 2) + (self.a(i, j) / self.dx)) 
    
    def D(self, i, j):
        return - self.dt * self.d(i, j)
    

    def solve_grid(self):
        """ Method for solving the mesh grid"""
        # 1. Compute all grid points for the last row
        self.grid[self.imax, :] = [self.t_up(j) for j in range(self.jmax + 1)]

        # 2. Iterate for all i from self.imax to 0 inclusive and update grid points:
        for i in range(self.imax, 0, -1):

            # 2.1 (i, 0) boundary conditions for x_low
            self.grid[i - 1, 0] = self.x_low(i - 1)
            # 2.2 (i, -1) boundary conditions for x_up
            self.grid[i - 1, -1] = self.x_up(i - 1)

            # 2.3 v_(i-1, j) formula
            #     Keep in mind, this formula is applied to every row EXCEPT for the
            #     top and bottom row which was already defined by boundary conditions
            self.grid[i-1, 1:self.jmax] = [(self.A(i, j) * self.grid[i, j-1] 
                                            + self.B(i, j) * self.grid[i, j]
                                            + self.C(i, j) * self.grid[i, j+1]
                                            + self.D(i, j)) for j in range(1, self.jmax)]

In [2]:
from options import OptionsBlackScholes, American, Barrier

In [3]:
spot_init = 100  # Initial Spot price S_0
strike    = 100  # Strike price
rate      = 0.05 # Risk-free rate
tau       = 1/12  # Time to maturity
spot_low  = 0
spot_up   = 2 * spot_init
sigma     = 0.2

EuPut = OptionsBlackScholes(strike, rate, tau, sigma, (spot_low, spot_up), 'put')

EuPut_explicit = ExplicitScheme(EuPut, 100, 100)
EuPut_explicit.solve_grid()

print(f"Eu Put Option's Explicit Value: {EuPut_explicit.interpolate(0,80)}")

Eu Put Option's Explicit Value: 19.584377490573264


In [5]:
spot_init = 100  # Initial Spot price S_0
strike    = 100  # Strike price
rate      = 0.05 # Risk-free rate
D         = 118  # Barrier
tau       = 6/12 # Time to maturity
spot_low  = 0
spot_up   = 2 * spot_init
alpha     = 0.25 # alpha

sigma     = lambda t, s: 0.13 * math.exp(-t) * ((100/s)**alpha) # Lambda function representing std


upandoutcall = Barrier(strike, rate, tau, sigma, D, (spot_low, spot_up), 'up-out', 'call')


# upandoutcall_explicit = ExplicitScheme(upandoutcall, 100, 100)
# upandoutcall_explicit.solve_grid()

# print(f"Eu Put Option's Explicit Value: {upandoutcall_explicit.interpolate(0,80)}")

{'tau': 0, 'x_low': 0.0, 'x_up': 0.0}

In [16]:
upandoutcall.get_conditions(tau, spot_init)

{'tau': 0, 'x_low': 0.0, 'x_up': 0.0}

In [23]:
# Initialising variables for the pde
spot_init = 100  # Initial Spot price S_0
K         = 100  # Strike price
r         = 0.02 # Risk-free rate
D         = 118  # Barrier
T         = 0.5  # Time to maturity
alpha     = 0.25 # alpha

sigma     = lambda t, s: 0.13 * math.exp(-t) * ((100/s)**alpha) # Lambda function representing std

spot_low  = 0
spot_up   = 2 * spot_init

# Initialising up and out pde
pde = UpOutEuCallBS(r, sigma, K, D, T, spot_low, spot_up)

# Executing explicit scheme
explicit = ExplicitScheme(pde, 100, 100)
explicit.solve_grid() # Computing grid

# Interpolating for S = S_0
explicit_val = explicit.interpolate(0,100)
print(f"Explicit Value: {explicit_val}")

AM Put Option's Explicit Value: 19.007731778978236


In [19]:
AmPut_explicit = ExplicitScheme(AmPut, 100, 100)
AmPut_explicit.solve_grid()

print(f"European Put Option's Explicit Value: {AmPut_explicit.interpolate(0,80)}")

# Adjust the grid for the American option's early exercise feature
grid = AmPut_explicit.get_grid()
AmPut_explicit.grid = AmPut.adjust_grid(grid, AmPut_explicit.dx)

# Re-solve the grid using the adjusted values
AmPut_explicit.solve_grid()

print(f"American Put Option's Explicit Value: {AmPut_explicit.interpolate(0,80)}")

European Put Option's Explicit Value: 19.007731778978236
American Put Option's Explicit Value: 19.007731778978236


In [96]:
import math
from scipy.stats import norm

# Given variables

spot_init = 100  # Initial Spot price S_0
strike    = 100  # Strike price
rate      = 0.05 # Risk-free rate
tau       = 6/12  # Time to maturity
spot_low  = 0
spot_up   = 2 * spot_init
sigma     = 0.2

# Black-Scholes formula for European put option
def black_scholes_put_price(S, K, r, tau, sigma):
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * tau) / (sigma * math.sqrt(tau))
    d2 = d1 - sigma * math.sqrt(tau)
    
    put_price = K * math.exp(-r * tau) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put_price

european_put_price = black_scholes_put_price(spot_init, strike, rate, tau, sigma)
european_put_price

4.41971978051388